In [1]:
import pandas as pd

In [5]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [16]:
totalNumberOfSchools = school_data["school_name"].count()
totalNumberOfStudents = student_data["student_name"].count()
totalBudget = "${:0,.2f}".format(school_data["budget"].sum())
averageMathScore = student_data["math_score"].mean()
averageReadingScore = student_data["reading_score"].mean()
totalNumberOfStudents

39170

In [17]:
students_passing_math = student_data[student_data.math_score >= 70]
student_passing_math_count = students_passing_math["student_name"].count()
student_passing_math_count

29370

In [18]:
percentPassingMath = student_passing_math_count/totalNumberOfStudents * 100
percentPassingMath

74.9808526933878

In [20]:
students_passing_reading = student_data[student_data.reading_score >= 70]
student_passing_reading_count = students_passing_reading["student_name"].count()
student_passing_reading_count

33610

In [21]:
percentPassingReading = student_passing_reading_count/totalNumberOfStudents * 100
percentPassingReading

85.80546336482001

In [30]:
over_all_passing_df = student_data.where((student_data.math_score >= 70) & (student_data.reading_score >= 70))
over_all_passing_count = over_all_passing_df["student_name"].count()
over_all_passing_percent = over_all_passing_count/totalNumberOfStudents * 100
over_all_passing_percent

65.17232575950983